# Problem set 2

## Team
Please write here your names and team number.

* Team name:
* Team members:

## Using Colab with GitHub
To utilize GPU support for model training, we highly recommend to open this notebook with Google Colab. Simply, change the domain from 'github.com' to 'githubtocolab.com' and refresh the site to open the notebook in Colab.
If you haven't used Colab before with private repositories, make sure to grant Colab access to your private repositories (see screenshot) and after that just try to change the domain again.

Finally, you should make sure that you add a GPU to your Colab notebook. You can do so by clicking on `Runtime` →  `Change runtime type` → `Hardware accelerator`  →  `GPU`.

## Submission

Make sure that you always commit and push the changes you make in Colab back to GitHub. To do so from within a Colab notebook, click `File` → `Save a copy in GitHub`. You will be prompted to add a commit message, and after you click OK, the notebook will be pushed to your repository. Only changes that are visible in your GitHub repository on the main branch will be considered for grading. If you close Colab in your browser without pushing your changes to GitHub or saving them on Google Drive, they will be lost.

Make sure that all your work has been pushed to GitHub before the deadline.


Check that the GPU  enabled in your colab notebook by running the cell below.

In [ ]:
import torch
# Check is GPU is enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

# Get specific GPU model
if str(device) == "cuda:0":
  print("GPU: {}".format(torch.cuda.get_device_name(0)))

You will be working with the EuroSAT dataset. The dataset contains 8489 pictures of 3 different land coverage types (crop, herbaceous vegetation and river). Running the lines below will download the data and return a random picture from the dataset.

In [ ]:
from torchvision.datasets import EuroSAT
import os
import numpy as np

data = EuroSAT(root=os.getcwd(), download=True) #downloads the dataset to your current directory
print(f"The dataset has {len(data)} images")
randint = np.random.randint(len(data))

pic, tar = data[randint]
print(f"Picture number {randint} with label: {tar}")
pic

# Task 1: Transform the data (10 pt)

 Your task is to train a classifier to classify the different land usage types in the dataset. We want to select only the 50 most frequent people in the dataset, all the other people should be mapped to a common class.
- Implement the class `rotate` that maps pictures to flipped pictures by 90, 180, 270 or 360°. The class should return an error if you try to rotate the picture by other degrees.
- Plot a histogram with the frequencies of each class. Make sure to insert both name and label in the histogram (e.g. `AnnualCrop:0`).
- We create a class `RotateEuroSAT` that takes as input the original dataset and returns a new dataset which contains randomly rotated pictures and whose label proportion can be customized.
Implement the class method `_create_rotated_dataset` that returns this pictures using the previously implemented `rotate`.
- `RotateEuroSAT` should also take care of transforming the pictures to tensors and optionally move the tensor to a GPU device.

In [ ]:
import random
from torch.utils.data import Subset, Dataset, random_split
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from collections import Counter

def rotate_picture(picture, rotation):
  """
  Rotate the image by the given degrees: 90, 180, 270, or 360.

  :param picture: Image to rotate
  :param rotation: Rotation in degrees
  :return: Rotated image
  """

  # raise error
  if rotation not in [90, 180, 270, 360]:
    raise ValueError("Rotation angle should be 90, 180, 270, or 360")

  return picture.rotate(rotation)


def plot_histogram(data):
  """

  Plot a histogram with the frequencies of each class.

  data: EuroSAT dataset
  return: fig, ax
  """

  fig, ax = plt.subplots(figsize=(10, 6))

  # Extract labels from dataset
  labels = [item[1] for item in data]

  # Get class names from the dataset
  classes_names = data.classes

  # calculate frequencies of each class
  label_counts = Counter(labels)

  # prepare data for plotting
  label_indices = list(label_counts.keys())
  counts = list(label_counts.values())
  class_labels = [classes_names[idx] for idx in label_indices]  # Map labels to class names


  # plot the histogram
  ax.bar(label_indices, counts)
  ax.set_xlabel('Class')
  ax.set_ylabel('Frequency')
  ax.set_title('Class Frequency Histogram')

  # annotating each bar with the count
  for i, count in enumerate(counts):
    ax.text(label_indices[i], count + 0.1, f"{class_labels[i]}:{label_indices[i]}",
            ha='center',rotation=45, fontsize=9)

  return fig, ax


new_pic = rotate_picture(pic, 90) # Example of rotating a picture by 90 degrees
same_pic = rotate_picture(pic, 360) # Example of rotating a picture by 360 degrees (should return the same picture)
fig, ax = plot_histogram(data)
fig.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

axes[0].imshow(pic)
axes[0].set_title("Original picture")
axes[1].imshow(new_pic)
axes[1].set_title("Rotated by 90°")
axes[2].imshow(same_pic)
axes[2].set_title("Rotated by 360°")

In [ ]:
class RotateEuroSAT(Dataset):
    """
    Initialize a rotated EuroSAT dataset.

    This dataset takes an existing EuroSAT dataset and applies random rotations (90°, 180°, 270°, or 360°)
    to the images, and splits them based on the provided class distribution (shares).

    :param original_data: The original EuroSAT dataset.
    :param length: The desired length of the new dataset (i.e., total number of images after rotation).
    :param shares: The distribution shares for each class. The list must sum to 1, and its length must match
                    the number of classes in the original dataset.
    :param device: The device (CPU or GPU) to move the images to. Default is None (images will stay on the CPU).
    :param seed: The random seed for reproducibility. Default is 42.
    """
    def __init__(self,
                 original_data: Dataset,
                 length: int,
                 shares: list,
                 device=None,
                 seed=42):
        self.original_data = original_data
        self.length = length
        assert sum(shares) == 1, "Shares must sum to 1"
        assert len(shares) == len(original_data.classes), "Shares must match number of classes"
        self.shares = shares
        self.seed = seed
        self.device = device

        self.transform = transforms.Compose([
            transforms.ToTensor(),
        ])

        self.dataset = self._create_rotated_dataset()

    def __len__(self):
        """
        Returns the length of the rotated dataset.

        :return: The length of the dataset (i.e., the number of images in the rotated dataset).
        :rtype: int
        """
        return self.length

    def __getitem__(self, idx):
        """
        Get an item (rotated image and its label) from the dataset.

        :param idx: The index of the item to fetch from the dataset.
        :type idx: int
        :return: A tuple of the rotated image (tensor) and the label.
        :rtype: tuple (torch.Tensor, int)
        """
        picture, label = self.dataset[idx]

        # Apply transformation to tensor
        picture = self.transform(picture)

        # Optionally move the tensor to the specified device (CPU/GPU)
        if self.device is not None:
            picture = picture.to(self.device)

        return picture, label

    def _create_rotated_dataset(self):
        """
        Create a dataset with rotated images based on the given shares for each class.

        This method rotates images from the original dataset randomly by one of the following angles:
        90°, 180°, 270°, or 360°.

        :return: A list of rotated images and their corresponding labels.
        :rtype: list of tuple (PIL.Image, int)
        """
        random.seed(self.seed)
        rotated_dataset = []

        # Set a start index
        start_idx = 0

        # Create list of the number of each class
        labels = [item[1] for item in self.original_data]
        label_counts_lst = list(Counter(labels).values())

        for class_idx, class_num in enumerate(label_counts_lst):
            # Calculate the number of images for this class based on the share
            class_share = self.shares[class_idx]
            num_class_images = int(class_share * self.length)
            class_images = [i for i in range(start_idx, start_idx + class_num) if self.original_data[i][1] == class_idx]

            # Rotate images
            for img_idx in class_images:
                image, label = self.original_data[img_idx]

                # Rotate the image by one of the allowed angles (90, 180, 270, 360)
                rotation_angle = random.choice([90, 180, 270, 360])
                rotated_image = image.rotate(rotation_angle)

                # Add rotated image and label to the new dataset
                rotated_dataset.append((rotated_image, label))

            start_idx += class_num

        return rotated_dataset




In [ ]:
rotated_data = RotateEuroSAT(data,
                             length=10**4,
                             shares=[1 / len(data.classes) for _ in data.classes],
                             device=device,
                             seed=42,)

train_data, test_data = random_split(rotated_data, [0.8, 0.2])

## Task 2: Implement a max pooling class and a CNN model(15 pt)
Implement a classification model to predict the label of the faces in the dataset. You are free to experiment with the network architecture. However your model **must** contain:
- At least one max pooling layer, implemented with `MyMaxPool`,
- Convolutional, linear, and pooling layers only,
- At least 3 convolutional layers, with at least two different kernel sizes,
- A final output layer that is customizable to the number of classes that we want to predict.

#### Briefly explain why you chose the particular architecture you implemented (around 2-3 sentences).


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms

class MyMaxPool(nn.Module):
    def __init__(self, kernel_size=2, stride=2):
        super(MyMaxPool, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self,x):
        return nn.functional.max_pool2d(x,self.kernel_size, self.stride)



class MyCNNModel(nn.Module):
    def __init__(self, num_classes: int = 10):
        """
        Initialize a CNN model with lazy layers that infer input dimensions
        automatically.

        Args:
            num_classes (int): The number of output classes.
        """
        super(MyCNNModel, self).__init__()

        # First Lazy Convolutional Layer (input channels will be inferred)
        self.conv1 = nn.LazyConv2d(out_channels=16, kernel_size=3, stride=1, padding=1)

        # Second Lazy Convolutional Layer (input channels will be inferred)
        self.conv2 = nn.LazyConv2d(out_channels=32, kernel_size=5, stride=1, padding=2)

        # Third Lazy Convolutional Layer (input channels will be inferred)
        self.conv3 = nn.LazyConv2d(out_channels=64, kernel_size=3, stride=1, padding=1)

        # Max Pooling Layer
        self.pool = MyMaxPool(kernel_size=2, stride=2)

        # Fully connected layer will be created lazily (input size inferred)
        self.fc1 = nn.LazyLinear(out_features=64)  # Output will be inferred during the first forward pass
        self.fc2 = nn.Linear(in_features=64, out_features=num_classes)  # The final output layer based on the number of classes

        # ReLU activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Forward pass through convolutional layers with ReLU activation and pooling
        x = self.relu(self.conv1(x))  # First conv layer
        x = self.pool(x)  # Pooling

        x = self.relu(self.conv2(x))  # Second conv layer
        x = self.pool(x)  # Pooling

        x = self.relu(self.conv3(x))  # Third conv layer
        x = self.pool(x)  # Pooling

        # Flatten the output before passing to fully connected layers
        x = x.flatten(start_dim=1)

        # Fully connected layers with ReLU activation
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Final output layer

        return x

In [ ]:
train_x, train_y = train_data[0]

In [ ]:
#print one iteration of your model to test its correctness

my_model = MyCNNModel(num_classes=10).to(device)  # 10 classes for EUROSET database
output = my_model(train_x)
print(" Model test for correctness")
print("Input shape:", train_x.shape)
print("Output shape:", output.shape)



## Training

We define a `Trainer` function to train our model that returns avg loss and avg accuracy per epoch. We set the configuration of the trainer is set in the `cfg` dictionary. Use the trainer to train your model and make sure to print and plot avg loss and accuracy using the in-built commands.

In [ ]:
import os, datetime, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm

cfg = {
    'batch_size': 64,
    'criterion': 'CrossEntropyLoss', #change to 'nn.NLLLoss' if you are applying a softmax in the last layer of your model
    'epochs': 50,
    'learning_rate': 0.001,
    'optimizer':'Adam',
    'seed':42,

}

class Trainer:
    """
    Trainer class for training, evaluating, and logging the model's performance.
    """

    def __init__(self,
                 model: torch.nn.Sequential,
                 cfg:dict,
                 device = None,
                 ):
        """
        Initialize.

        :param model: The model to be trained.
        :param cfg: A dictionary containing hyperparameters such as batch size, optimizer, etc.
        :param device: The device (CPU or GPU) on which the model will be trained. Defaults to None.
        """
        self.model = model
        self.cfg = cfg
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)  # Move the model to the specified device

        # lists to store loss and accuracy for plotting
        self.all_avg_acc=[]
        self.all_avg_loss=[]

    def torch_train(self, data, shuffle=False):
        """
        Train model

        :param data: train data
        :param shuffle: boolean whether shuffle or not.
        :return: trained model
        """
    # Get the loss function (criterion) based on the provided config
        criterion = getattr(nn, self.cfg['criterion'])()  # CrossEntropyLoss, etc.

        # Initialize the optimizer based on the provided config
        optimizer = getattr(optim, self.cfg['optimizer'])(
            self.model.parameters(), lr=self.cfg['learning_rate']
        )

        # Set the model to training mode
        self.model.train()

        # Create DataLoader for the training data
        dataloader = DataLoader(data,
                                batch_size=self.cfg['batch_size'],
                                shuffle=shuffle)

        running_loss = 0
        correct_preds = 0
        total_preds = 0
        for epoch in range(self.cfg['epochs']):

          for x, y in tqdm(dataloader):  # tqdm is used to display a progress bar
            x = x.to(self.device)  # move input data to the device (GPU/CPU)
            y = y.to(self.device)  # move target labels to the device (GPU/CPU)


            # zero the gradients before the forward pass
            optimizer.zero_grad()

            # forward pass: get model predictions
            pred = self.model(x)

            # calculate the loss
            loss = criterion(pred.squeeze(-1),y.long())

            # backward pass: calculate gradients
            loss.backward()

            # update model parameters based on gradients
            optimizer.step()

            # update running loss
            running_loss += loss.item() * x.size(0)

            # calculate accuracy: compare predicted and true labels
            _, predicted = torch.max(pred, 1)
            correct_preds += (predicted == y).sum().item()
            total_preds += y.size(0)

          # calculate average loss and accuracy for this epoch
          avg_loss = running_loss / total_preds
          avg_acc = correct_preds / total_preds
          self.all_avg_acc.append(avg_acc)
          self.all_avg_loss.append(avg_loss)

        return self.model
    def plot_training_metric(self):
        """
        Plot the training loss and accuracy per epoch.
        """
        plt.figure(figsize=(12, 5))

        # Plot loss
        plt.subplot(1, 2, 1)
        plt.plot(self.all_avg_loss, label='Train Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Loss')

        # Plot accuracy
        plt.subplot(1, 2, 2)
        plt.plot(self.all_avg_acc, label='Train Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Training Accuracy')

        plt.show()

    def evalute_test(self, model, test_data):
        """
        evalute model with test data

        :param model: trained model
        :param test_data: test data
        :return: avg accuracy
        """
        # create DataLoader for the test data
        dataloader = DataLoader(test_data,
                                batch_size=self.cfg['batch_size'],
                                shuffle=False)
        # get a batch of test data
        x_test, y_test = next(iter(dataloader))
        x_test = x_test.to(self.device)
        y_test = y_test.to(self.device)

        # get model predictions
        pred = model(x_test)

        # get the predicted class with the highest probability
        y_pred = pred.argmax(dim=-1)

        # calculate the average accuracy
        avg_acc = (y_pred == y_test).float().mean().item()

        return avg_acc





In [ ]:
# Assuming `train_data` and `val_data` are already defined and are instances of DataLoader
my_model = MyCNNModel(num_classes=10).to(device)
trainer = Trainer(model=my_model, cfg=cfg, device=device)
train_model = trainer.torch_train(train_data)

In [ ]:
trainer.plot_training_metric()

In [ ]:
print(f"Test Accurcay: {trainer.evalute_test(train_model, test_data)}")

## Task 3: Tune your training hyperparameters (optional, 10 pt)

Implement a method <code>grid_search</code>, which looks for the best possible learning rates and training batch sizes for your model <code>MyCNNModel</code> and returns the best possible model, the corresponding training configuration, and the final training avg losses and accuracies (as numbers).

In [ ]:
def grid_search(train_dataset, cfg, learning_rates=[10**-1, 10**-2, 10**-3], batch_sizes=[2**5, 2**6, 2**7]):
    '''#TODO: here your code '''
    return best_model, best_cfg, best_avg_loss, best_avg_acc


best_model, best_cfg, best_avg_loss, best_avg_acc = grid_search(train_data, cfg, learning_rates=[10**-1, 10**-2, 10**-3], batch_sizes=[2**5, 2**6, 2**7])
print(f"Best model achieves {best_avg_loss:.2f} loss and {best_avg_acc:.1%} accuracy").


## Task 3: Load and fine-tune a pre-trained model (10 pt)

<ul>
  <li>Load and train a pre-trained model for classification problems, such as those made available in <a href="https://huggingface.co/docs/timm">Hugging Face's timm library</a>. </li>
  <li> Make sure to modify the output layer to be compatible with the number of classes. </li>
  <li>Print a summary of your results.</li>
  <li>Briefly explain why you chose the particular architecture you did (around 2-3 sentences).</li>
  </ul>
  
<b>Note</b>: in case you run into computing-related (e.g. memory) issues, consider choosing another model.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import transforms, models
from torchvision.datasets import EuroSAT
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Set random seeds for reproducibility
def set_seed(seed=42):
    """Set random seeds for reproducible results"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

def create_data_transforms():
    """
    Create data transformation pipelines for training and validation.
    """

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomRotation(degrees=45),
        transforms.ColorJitter(
            brightness=0.3,
            contrast=0.3,
            saturation=0.3,
            hue=0.1
        ),
        transforms.ToTensor(),
        transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.33))
    ])

    val_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])

    return train_transform, val_transform


In [ ]:
# Load your existing data using the code you already have
# This is the code you provided earlier
from torchvision.datasets import EuroSAT
data = EuroSAT(root=os.getcwd(), download=True)
print(f"The dataset has {len(data)} images")

# Get number of classes and class names
num_classes = len(data.classes)
class_names = data.classes
print(f"Number of classes: {num_classes}")
print(f"Classes: {', '.join(class_names)}")

# Create train/val/test split
total_size = len(data)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Random split
train_dataset, val_dataset, test_dataset = random_split(
    data, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"\nDataset split:")
print(f"  Training:   {len(train_dataset)} samples")
print(f"  Validation: {len(val_dataset)} samples")
print(f"  Test:       {len(test_dataset)} samples")

# Apply transforms
train_transform, val_transform = create_data_transforms()

# Create new datasets with transforms
train_data_transformed = EuroSAT(root=os.getcwd(), download=False, transform=train_transform)
val_data_transformed = EuroSAT(root=os.getcwd(), download=False, transform=val_transform)
test_data_transformed = EuroSAT(root=os.getcwd(), download=False, transform=val_transform)

# Apply the same indices from the split
train_dataset = Subset(train_data_transformed, train_dataset.indices)
val_dataset = Subset(val_data_transformed, val_dataset.indices)
test_dataset = Subset(test_data_transformed, test_dataset.indices)

In [ ]:

batch_size = 32 if torch.cuda.is_available() else 16
num_workers = min(4, os.cpu_count() // 2) if os.cpu_count() else 0

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available()
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available()
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=torch.cuda.is_available()
)

print(f"\nDataLoaders created with batch size: {batch_size}")


In [ ]:

def create_resnet50_model(num_classes, dropout_rate=0.5):
    """
    Create a ResNet50 model with a custom classifier head.
    """

    print(f"\nCreating ResNet50 model...")

    # Load pretrained ResNet50
    model = models.resnet50(pretrained=True)
    print("✓ Loaded pretrained ImageNet weights")

    # Get the number of features from the last layer
    num_features = model.fc.in_features  # Should be 2048 for ResNet50

    # Replace the final fully connected layer with custom classifier
    model.fc = nn.Sequential(
        nn.BatchNorm1d(num_features),
        nn.Dropout(dropout_rate),
        nn.Linear(num_features, 512),
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(512),
        nn.Dropout(dropout_rate * 0.7),
        nn.Linear(512, 256),
        nn.ReLU(inplace=True),
        nn.BatchNorm1d(256),
        nn.Dropout(dropout_rate * 0.5),
        nn.Linear(256, num_classes)
    )

    print(f"✓ Replaced classifier head for {num_classes} classes")

    # Initialize the new layers
    for m in model.fc.modules():
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

    return model

# Create the model
model = create_resnet50_model(num_classes=num_classes, dropout_rate=0.5)

In [ ]:

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Update progress bar
        pbar.set_postfix({
            'loss': running_loss / (pbar.n + 1),
            'acc': 100. * correct / total
        })

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

def validate(model, val_loader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            pbar.set_postfix({
                'loss': running_loss / (pbar.n + 1),
                'acc': 100. * correct / total
            })

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

def test_accuracy(model, test_loader, device):
    """Get test accuracy"""
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100. * correct / total

In [ ]:
print("STAGE 1: FROZEN BACKBONE TRAINING (10 EPOCHS)")
print("="*60)

# Move model to device
model = model.to(device)

# FREEZE all layers except the classifier (fc)
for name, param in model.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False

print("✓ Frozen all layers except classifier")

# Count trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.1f}%)")

# Setup training
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=0.001,
    weight_decay=1e-4
)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'test_acc': [],
    'lr': []
}

# Best model tracking
best_val_loss = float('inf')
best_model_state = None

# Train for 10 epochs
for epoch in range(10):
    print(f"\n--- Epoch {epoch+1}/10 ---")

    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)

    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    # Test
    test_acc = test_accuracy(model, test_loader, device)

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict().copy()
        print("  ⭐ New best model saved!")

    # Update scheduler
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']

    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['test_acc'].append(test_acc)
    history['lr'].append(current_lr)

    # Print summary
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f}, Val Acc:   {val_acc:.2f}%")
    print(f"  Test Acc:   {test_acc:.2f}%")
    print(f"  Learning Rate: {current_lr:.6f}")

# Load best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("\n✓ Loaded best model based on validation loss")

# Final test accuracy
final_test_acc = test_accuracy(model, test_loader, device)
print(f"\n📊 Final Test Accuracy: {final_test_acc:.2f}%")


In [ ]:
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': 10,
    'train_history': history,
    'best_val_loss': best_val_loss,
    'final_test_acc': final_test_acc,
    'num_classes': num_classes,
    'class_names': class_names
}

checkpoint_path = 'stage1_frozen_backbone_checkpoint.pth'
torch.save(checkpoint, checkpoint_path)
print(f"✓ Model checkpoint saved to: {checkpoint_path}")

In [ ]:

def plot_stage1_results(history):
    """Plot Stage 1 training results"""

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    epochs = range(1, len(history['train_loss']) + 1)

    # Plot 1: Training Loss vs Epoch
    axes[0].plot(epochs, history['train_loss'], 'b-', linewidth=2, marker='o', markersize=4)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Training Loss vs Epoch (Stage 1)', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xticks(range(1, 11))

    # Plot 2: Training Accuracy vs Epoch
    axes[1].plot(epochs, history['train_acc'], 'g-', linewidth=2, marker='s', markersize=4)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy (%)', fontsize=12)
    axes[1].set_title('Training Accuracy vs Epoch', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_xticks(range(1, 11))
    axes[1].set_ylim([min(history['train_acc'])*0.95, 100])

    # Plot 3: Test Accuracy vs Epoch
    axes[2].plot(epochs, history['test_acc'], 'r-', linewidth=2, marker='o', markersize=5)
    axes[2].set_xlabel('Epoch', fontsize=12)
    axes[2].set_ylabel('Accuracy (%)', fontsize=12)
    axes[2].set_title('Test Accuracy vs Epoch', fontsize=14, fontweight='bold')
    axes[2].grid(True, alpha=0.3)
    axes[2].set_xticks(range(1, 11))
    axes[2].set_ylim([min(history['test_acc'])*0.95, max(history['test_acc'])*1.02])

    # Annotate best test accuracy
    best_test_acc = max(history['test_acc'])
    best_epoch = history['test_acc'].index(best_test_acc) + 1
    axes[2].annotate(f'Best: {best_test_acc:.2f}%',
                    xy=(best_epoch, best_test_acc),
                    xytext=(best_epoch+1, best_test_acc-1),
                    arrowprops=dict(arrowstyle='->', color='red', alpha=0.7),
                    fontsize=10, color='red')

    plt.suptitle('Frozen Backbone Training Results', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

# Plot the results
plot_stage1_results(history)


In [ ]:
print("\n" + "="*60)
print("STAGE 1 TRAINING COMPLETE - SUMMARY")
print("="*60)
print(f"""
📊 Training Results:
   - Total Epochs: 10
   - Final Test Accuracy: {final_test_acc:.2f}%
   - Best Test Accuracy: {max(history['test_acc']):.2f}%
   - Best Validation Loss: {best_val_loss:.4f}
   - Improvement: {history['test_acc'][-1] - history['test_acc'][0]:.2f}%

""")